In [1]:
import geopandas as gpd

In [2]:
HD72_MUNICIPALITIES = [
    'JOHNSTOWN CITY',
    'BLACKLICK TWP',
    'CAMBRIA TWP',
    'CONEMAUGH TWP',
    'CROYLE TWP',
    'EAST TAYLOR TWP',
    'JACKSON TWP',
    'LOWER YODER TWP',
    'MIDDLE TAYLOR TWP',
    'UPPER YODER TWP',
    'WEST TAYLOR TWP',
    'BROWNSTOWN BORO',
    'DAISYTOWN BORO',
    'DALE BORO',
    'EAST CONEMAUGH BORO',
    'EBENSBURG BORO',
    'EHRENFELD BORO',
    'FRANKLIN BORO',
    'LORAIN BORO',
    'NANTY GLO BORO',
    'SOUTHMONT BORO',
    'SUMMERHILL BORO',
    'VINTONDALE BORO',
    'WESTMONT BORO'
]

# Generate GeoJSON map

### Standardize township class, limit municipalities to those in the district

In [3]:
munis = gpd.read_file('./input/municipalities-pa.geojson')

munis.loc[munis.CLASS_OF_M.isin(['1TWP', '2TWP']), 'CLASS_OF_M'] = 'TWP'
munis['muni_name'] = munis.MUNICIPAL1 + ' ' + munis.CLASS_OF_M

munis = munis[(munis.FIPS_COUNT == '021') & (munis.muni_name.isin(HD72_MUNICIPALITIES))]
munis.to_file('./output/municipalities-hd72.geojson', driver='GeoJSON')